#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, as the global variables of interest in order to associate it with the output name file.

In [360]:
import pandas as pd
import numpy as np
import copy
from __future__ import division
import minimization as minz #My script in same folder for minimization routines


#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "trial1"
n_states = 4
n_signals = 0
n_rounds = 20
N = 40
n_parents = 20

globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_to_text/" #Netlogo outputs in this folder

#Read files and save them as data
df_sum = pd.read_csv(netlogo_folder + summary_file_name)
df_strat = pd.read_csv(netlogo_folder + strategies_file_name)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [70]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

canon_autos_list = [] #Create empty lists to store the processed autos below
min_autos_list = []


for i in df_strat.index: #For each row...
    netlogo_auto = df_strat.auto_long[i] #... for all netlogo_auto
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always 0 for canonical auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #Update autos lists
    canon_autos_list.append(canon_auto) #Store proccessed autos in the corresponding list
    min_autos_list.append(min_auto)
    
#Add the processed autos lists as columns to df_strat
df_strat["canon_autos"] = canon_autos_list #Add the lists with autos to the dataframe
df_strat["min_autos"] = min_autos_list

#3)  Variables key for analysis

##3.1 Accesible states and minimum behavioural states

In [71]:
#Accesible states

df_strat["access_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.canon_autos[i]) #Accesible states is the lenght of the canonical auto
    df_strat.loc[i, "access_states"] = states #Save to dataframe

#Minimum behavioral states

df_strat["min_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.min_autos[i]) #Min states is the lenght of the min auto, taken as proxy for complexity
    df_strat.loc[i, "min_states"] = states #Save to dataframe

##3.2 Joint machine (not minimized) and Unused States measure (WORKS ONLY WITHOUT SIGNAL)

In [72]:
#Prepare dataframe to keep track of unused states
df_strat["used_states"] = 0 # Will contain a list with states of each min_auto
used_states_list = []
for i, auto in enumerate(df_strat.min_autos): #all minimized autos
    a = [0 for ix in xrange(df_strat.min_states[i])] #List the size of minimised machine's states
    used_states_list.append(a)
df_strat["used_states"] = used_states_list #Add to dataframe

# Lists to keep track of joint machines
gen_list = []
jm_list = []

#Create the joint machines (not minimized) and track unused states
#Main code of this part

for gen in df_sum.generation: #each generation
    print "generation = ", gen #Useful for debugging (or keeping track of simulation time)
    #Next line (the "for" block) is tricky:
    #It creates a dataframe containing only the autos with population = column and current generation.
    #Uses "iterrows()" to iterate over the index of the dataframe (df_strat), keeping it on i0, which is
    #needed to acces later the particular auto that was used for the joint machine (accesed by row0.min_autos)
    #This is used (instead of a simple enumerate) to access that row later when updating the used states 
    #Same logic for the second "for" block, but for population row
    
    for i0, row0 in df_strat[(df_strat.population == "column") & (df_strat.generation == gen)].iterrows():
        for i1, row1 in df_strat[(df_strat.population == "row") & (df_strat.generation == gen)].iterrows():
            
            #print "generation = ", gen, "i0 = ", i0, "i1 = ", i1
            auto0 = row0.min_autos #Autos to pass to create_joint_machine function
            auto1 = row1.min_autos
            
            #Joint machine only work WITHOUT signal so far (change "create_joint_machine" and "update_state" functions)
            jm = minz.create_joint_machine_no_signal(auto0, auto1) #Function to create the joint machine
            
            gen_list.append(gen) #Keeps track of generation
            jm_list.append(jm)   #Keeps track of joint machines
            
            #Update used_states:
            #This part takes the information from the just create joint machine jm, which contains the states that
            #are visited by the two autos that created it. For each auto, then goes trough each state. The
            #list "used_states" contains a list of zeros with the number of states of the auto, each position
            #representing each state of the auto. So if state 1 of the machine is used in the joint machine, then
            #position 1 will be changed from zero to one, to indicate that the state is used.
            
            for st in jm["states"]: #for metastates in jointmachine
                s0 = st[0] #state of auto0
                df_strat["used_states"][i0][s0] = 1 # 1 for states visited. Unvisited remain 0
            
            for st in jm["states"]:
                s1 = st[1] #state of auto1
                df_strat["used_states"][i1][s1] = 1


#Store the joint machines and generation (a new dataframe)
df_jms = pd.DataFrame(columns = ("generation", "jm")) #Store joint machines with associated generation
df_jms.generation = gen_list
df_jms.jm = jm_list

generation =  0
generation =  1
generation =  2
generation =  3
generation =  4
generation =  5
generation =  6
generation =  7
generation =  8
generation =  9
generation =  10
generation =  11
generation =  12
generation =  13
generation =  14
generation =  15
generation =  16
generation =  17
generation =  18
generation =  19
generation =  20
generation =  21
generation =  22
generation =  23
generation =  24
generation =  25
generation =  26
generation =  27


##3.3 Minimize the joint machines (without signal)

In [157]:
min_jm_list = [] #to save the minimized joint machines, and add later to the dataframe (df_jms)

for jm in df_jms.jm: #all joint machines
#for jm in [df_jms.jm[24000], df_jms.jm[24001]]:
    min_jm = minz.minimize_joint_machine_no_signal(jm) #minimize them (have actions and cyclestart)
    #print min_jm
    
    #Next lines convert the min_jms, which is a dict, into a tuple
    #Basically converts a list of lists ("actions" in the min_jm)into a tuple of tuples
    #This is so that it can be used as a key to use groupby (since tuple is inmutable)
    tup_actions = tuple(tuple(pair_actions) for pair_actions in min_jm["actions"]) #convert actions to tuples
    min_jm = (tup_actions, min_jm["cyclestart"]) #add the cyclestart to final min_jm tuple
    
    min_jm_list.append(min_jm) #save the minimized machine to a list
    #print min_jm
    
# Save the minimized joint machines
df_jms["min_jms"] = None #new empty column in dataframe
df_jms.min_jms = min_jm_list #add the minimized joint machines to the dataframe



##3.4 Frequencies of minimized joint machines
The dataframe "freqjm" will contain the required information

In [413]:
g1 = df_jms.groupby([df_jms["generation"], df_jms["min_jms"]]) #split by groups on "generation" and each joint machine
freqjm = g1.count() #dataframe with frequency of joint machine per generation
interactions = N * N #number of joint machines per generation

freq_perc_list = [(x*100)/interactions for x in freqjm.jm] #list with frequency percentage of jm per generation
freqjm['freq_perc'] = freq_perc_list #add frequency to the dataframe
freqjm = freqjm.reset_index() #reset_index converts the multiindex into normal columns
threshold = 5 #don't show joint machines with less than threshold% per generation
freqjm = freqjm[freqjm.freq_perc > threshold] #keep machines with frequency higher than threshold
freqjm = freqjm.sort(['generation', 'freq_perc'], ascending=[True, False]) #sort
freqjm = freqjm.rename(columns = {'jm':'freq'}) #rename column

In [414]:
freqjm

,generation,min_jms,freq,freq_perc
150,0,"(((A, B),), 0)",126,7.8750
417,0,"(((B, B),), 0)",110,6.8750
608,1,"(((A, B),), 0)",160,10.0000
717,1,"(((B, B),), 0)",153,9.5625
546,1,"(((A, A),), 0)",84,5.2500
847,2,"(((B, B),), 0)",360,22.5000
779,2,"(((A, B), (B, A), (A, A)), 2)",150,9.3750
919,3,"(((B, B),), 0)",300,18.7500
901,3,"(((A, B), (B, A), (A, B), (A, A)), 3)",253,15.8125
898,3,"(((A, B), (B, A), (A, A)), 2)",250,15.6250


In [ ]:

#FOR EXPLAINING STUFF


# unused behavior measures (using used states list)
#joint machine frequencies
#other machines frequencies (minimised)
#regime identification (using joint machines)
#regime change identification
#Graphs and stats (automate them)

##Next:
#1) describe graphically his transition explanations